In [4]:
import os

## switch to the root dir

In [2]:
os.chdir('..')
os.listdir()

['.git',
 '.gitignore',
 '.idea',
 'data_loader',
 'data_processor',
 'jupyter',
 'loggers',
 'main.py',
 'models',
 'original_data',
 'processed_data',
 'README.md',
 'saved_models',
 'trainers']

In [3]:
from data_processor.utils import load_numpy_arrays, numpy_to_tensor
from models.CNN import CNN_2d_block
from models.MLP import MLP
import trainers.SGD_trainer
from loggers.statistics_loggers import plot_numerical_arrays
import torch
from torch import nn


OSError: [WinError 1455] 页面文件太小，无法完成操作。 Error loading "D:\Program\anaconda\lib\site-packages\torch\lib\cufft64_10.dll" or one of its dependencies.

## Load preprocessed data

In [ ]:
X_train, y_train, X_val, y_val = load_numpy_arrays(['data_train_X.npy','data_train_y.npy', 'data_val_X.npy','data_val_y.npy']
                  , path_prefix='processed_data/processed_waves/4-genres/')

In [ ]:
X_train, y_train, X_val, y_val = numpy_to_tensor([X_train, y_train, X_val, y_val])

In [ ]:
y_train = y_train.type(torch.int64)
y_val = y_val.type(torch.int64)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(X_train[0])
print(y_train[0])

In [ ]:
print(X_val.shape)
print(y_val.shape)

## Load model

1. define model under the folder models
2. import and define the model

In [ ]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        
        self.LSTM = nn.LSTM(input_size=20, hidden_size=40, num_layers=8, dropout=0.15, batch_first=True)
        self.MLP = MLP(40, 4, [100])
        
        
    
    def forward(self, x):
        out = x
        out, _ = self.LSTM(out)
        out = out[:, -1, :]
        out = self.MLP(out)
        return out

In [ ]:
# eg. MLP
model = LSTM()

In [ ]:
n_feat = X_train.shape[2]
n_time = X_train.shape[1]

In [ ]:
X_train = X_train[:, range(0, n_time, 2), :]

In [ ]:
X_a = X_train[:10].reshape((-1, 1, n_time, n_feat))
pred = model(X_a)

## Define loss and optimizer

In [ ]:
lr = 0.05
batch_size = 10
num_epoch = 10
model_name = 'CRNN'
saved_model_name = 'saved_models/saved_' + model_name + '_wave.pt'

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## Train

use trainers.SGD_trainer.train or define a trainer

parameters of SGD_trainer.train
- model
- train_array: a tuple (X_train, y_train, X_val, y_val)
- loss
- optimizer
- batch_size
- num_epoch
- device

In [ ]:
training_loss_array, training_accuracy_array, validation_loss_array, validation_accuracy_array = trainers.SGD_trainer.train(model, (X_train, y_train, X_val, y_val), 
                                                                                                                            loss, optimizer, batch_size=batch_size, num_epoch=num_epoch, device='cuda')

## Plot

plot_numerical_arrays: plot multiple arrays with the same length

parameters:
- num_arrays: numerical arrays with the same length
- labels: labels of each array(with the same order of num_arrays)
- xlabel
- ylabel
- title

In [ ]:
plot_numerical_arrays([training_loss_array, validation_loss_array], ['training loss', 'validation loss'], 
                      xlabel='batches', ylabel='loss', title='train and validation losses for logistic regression')

In [ ]:
plot_numerical_arrays([training_accuracy_array, validation_accuracy_array], ['training accuracy', 'validation accuracy'], 
                      xlabel='batches', ylabel='accuracy', title='train and validation accuracies for logistic regression')

## Save model

Save model in 'saved_models/saved_modelname_wave.pt'

In [ ]:
torch.save(model.state_dict(), saved_model_name)